In [2]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

# <font color='orange'>01. load data

In [3]:
rootpath = r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\temp'

In [4]:
os.listdir(rootpath)

['temp1_inner_total_df.csv', 'temp2_inner_total_df.csv', 'total_df.pkl']

In [5]:
total_df = pd.read_pickle(os.path.join(rootpath, 'total_df.pkl'), )

In [6]:
inner_total_df = pd.read_csv(os.path.join(rootpath, 'temp1_inner_total_df.csv'))

In [7]:
inner_total_df.head(2)

,no,신청월,직군그룹,근속기간1,웰컴_소득구간,심사원장_소득구간,EXECUTE_FG,DESTRUCT_FG,Y_2017_FG,Y_2018_FG,Y_2019_FG,DUP_FG_1,DUP_FG_2,FINAL_DUP_FG
0,1,201612,NaN,5.0,5.0,0.0,0,1,1,0,0,0,0,0
1,2,201612,D,5.0,2.0,0.0,0,1,1,0,0,0,0,0


# <font color='orange'>02. CB, SP 0등급 대상 선정

In [8]:
# 등급별 현황파악
print('CB등급별 건수')
print(total_df['Grd_RK0400_000'].value_counts().sort_index(),)
print('SP등급별 건수')
print(total_df['Grd_RK0400_700'].value_counts().sort_index())

CB등급별 건수
0         64
1       2045
2       4378
3       6415
4      19867
5      47846
6     132259
7     110913
8      63397
9      24028
10      7767
Name: Grd_RK0400_000, dtype: int64
SP등급별 건수
0         63
1       9663
2      41160
3      59542
4      86323
5     111644
6      52820
7      26160
8      14518
9      11887
10      5199
Name: Grd_RK0400_700, dtype: int64


In [9]:
# CB, SP 0등급 적중 함수 생성
def grade_zero(cb, sp):
    result = 0
    if cb==0 or sp==0:
        result = 1
        
    return result

In [10]:
# input columns list 생성
input_cols = ['Grd_RK0400_000', 'Grd_RK0400_700']

In [11]:
# CB, SP 0등급 항목 생성
total_df['GRADE_ZERO_FG'] = total_df[input_cols].apply(lambda x: grade_zero(x['Grd_RK0400_000'], x['Grd_RK0400_700']), axis = 1)

In [12]:
total_df['GRADE_ZERO_FG'].sum()

64

In [13]:
# 내부항목 dataframe 적용
inner_total_df['GRADE_ZERO_FG'] = total_df['GRADE_ZERO_FG']

In [14]:
inner_total_df['GRADE_ZERO_FG'].sum()

64

# <font color='orange'>03. 기부도 대상 선정

In [15]:
# def _pre_bad(BS0000145, BS0000172, BS0000169,
#             BS0000171, B12000200, B22000200,
#             PS0000296_1, P27000100):
    
#     BS0000145_FG = 0
#     BS0000172_FG = 0 
#     BS0000169_FG = 0
#     BS0000171_FG = 0
#     B12000200_FG = 0
#     B22000200_FG = 0
#     PS0000296_1_FG = 0
#     P27000100_FG = 0
    
#     if BS0000145 > 0:
#         BS0000145_FG = 1
        
#     if BS0000172 > 0:
#         BS0000172_FG = 1
        
#     if BS0000169 > 0:
#         BS0000169_FG = 1
        
#     if BS0000171 > 0:
#         BS0000171_FG = 1
        
#     if B12000200 > 0:
#         B12000200_FG = 1
        
#     if B22000200 > 0:
#         B22000200_FG = 1
        
#     if PS0000296_1 > 0:
#         PS0000296_1_FG = 1
        
#     if P27000100 > 0:
#         P27000100_FG = 1
        
#     return BS0000145_FG,BS0000172_FG, BS0000169_FG,BS0000171_FG,B12000200_FG,B22000200_FG,PS0000296_1_FG,P27000100_FG,
    
    
# total_df[result_cols] = total_df[need_cols].apply(lambda x: _pre_bad(x['BS0000145'],
#                                                                     x['BS0000172'],
#                                                                     x['BS0000169'],
#                                                                     x['BS0000171'],
#                                                                     x['B12000200'],
#                                                                     x['B22000200'],
#                                                                     x['PS0000296_1'],
#                                                                     x['P27000100'],), axis = 1, result_type = 'expand')

In [16]:
def pre_bad(col):
    result = 0
    if col>0:
        result=1
        
    return result

In [17]:
# output 항목명
result_cols = [ 'PRE_BAD_FG_1',
               'PRE_BAD_FG_2',
               'PRE_BAD_FG_3',
               'PRE_BAD_FG_4',
               'PRE_BAD_FG_5',
               'PRE_BAD_FG_6',
               'PRE_BAD_FG_7',
               'PRE_BAD_FG_8',]

In [18]:
# input 항목명
need_cols = [ 'BS0000145',
            'BS0000172', 
            'BS0000169',
            'BS0000171',
            'B12000200',
            'B22000200',
            'PS0000296_1',
            'P27000100',]

In [19]:
for in_col, out_col in zip(need_cols, result_cols):
    total_df[out_col] = total_df[in_col].apply(lambda x: pre_bad(x))

In [20]:
total_df[result_cols].sum()

PRE_BAD_FG_1     6350
PRE_BAD_FG_2     1479
PRE_BAD_FG_3     3040
PRE_BAD_FG_4     1834
PRE_BAD_FG_5    30888
PRE_BAD_FG_6     8025
PRE_BAD_FG_7     9231
PRE_BAD_FG_8    39734
dtype: int64

In [21]:
inner_total_df[result_cols] = total_df[result_cols]

### 최종 기부도 대상 항목 생성

In [22]:
# def _total_pre_bad(BS0000145_FG,BS0000172_FG, BS0000169_FG,BS0000171_FG,B12000200_FG,
#                    B22000200_FG,PS0000296_1_FG,P27000100_FG,):
#     total_pre_bad = 0
#     if (BS0000145_FG > 0 or BS0000172_FG > 0 or  BS0000169_FG > 0 or BS0000171_FG > 0 or B12000200_FG > 0 or
#                    B22000200_FG > 0 or PS0000296_1_FG > 0 or P27000100_FG > 0):
#         total_pre_bad = 1
        
#     return total_pre_bad

# total_df['TOTAL_PRE_BAD_FG'] = total_df[need_cols].apply(lambda x: _total_pre_bad(x['PRE_BAD_FG_1'],
#                                                                     x['PRE_BAD_FG_2'],
#                                                                     x['PRE_BAD_FG_3'],
#                                                                     x['PRE_BAD_FG_4'],
#                                                                     x['PRE_BAD_FG_5'],
#                                                                     x['PRE_BAD_FG_6'],
#                                                                     x['PRE_BAD_FG_7'],
#                                                                     x['PRE_BAD_FG_8'],), axis = 1,)

In [23]:
need_cols = [ 'PRE_BAD_FG_1',
               'PRE_BAD_FG_2',
               'PRE_BAD_FG_3',
               'PRE_BAD_FG_4',
               'PRE_BAD_FG_5',
               'PRE_BAD_FG_6',
               'PRE_BAD_FG_7',
               'PRE_BAD_FG_8',]

In [24]:
# 위에서 생성한 기부도 요건들을 or조건 적용
total_df['TOTAL_PRE_BAD_FG'] = total_df[need_cols].sum(axis=1)
total_df['TOTAL_PRE_BAD_FG'] = total_df['TOTAL_PRE_BAD_FG'].apply(lambda x: 1 if x>0 else 0)

In [25]:
inner_total_df['TOTAL_PRE_BAD_FG'] = total_df['TOTAL_PRE_BAD_FG']

In [26]:
inner_total_df.head(2)

,no,신청월,직군그룹,근속기간1,웰컴_소득구간,심사원장_소득구간,EXECUTE_FG,DESTRUCT_FG,Y_2017_FG,Y_2018_FG,...,GRADE_ZERO_FG,PRE_BAD_FG_1,PRE_BAD_FG_2,PRE_BAD_FG_3,PRE_BAD_FG_4,PRE_BAD_FG_5,PRE_BAD_FG_6,PRE_BAD_FG_7,PRE_BAD_FG_8,TOTAL_PRE_BAD_FG
0,1,201612,NaN,5.0,5.0,0.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,201612,D,5.0,2.0,0.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


# <font color='orange'>04. 저장

In [27]:
save_path = r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\temp'

In [28]:
inner_total_df.to_csv(os.path.join(save_path, 'temp2_inner_total_df.csv'), index = False)